In [ ]:
# 安裝必要套件
!pip install langgraph openai

In [ ]:
# 導入模組
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from openai import OpenAI
from typing import Dict
from google.colab import userdata

# 定義狀態結構
@dataclass
class SummaryState:
    input: str  # 輸入新聞
    summary: str = ""  # 生成的摘要
    post: str = ""  # 生成的X貼文
    iteration: int = 0  # 迭代次數

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 從Colab環境獲取金鑰

# 語言模型調用函數
def llm_call(prompt: str, max_tokens: int = 150) -> str:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.5  # 保持中立語氣
    )
    return response.choices[0].message.content.strip()

# 摘要生成節點
def summary_node(state: SummaryState) -> Dict:
    prompt = f"將以下1000字新聞摘要為100字，保留主要事件、影響，語氣中立，語句流暢：\n{state.input}"
    state.summary = llm_call(prompt)
    return {"summary": state.summary}

# 貼文生成節點
def post_node(state: SummaryState) -> Dict:
    prompt = f"將以下100字摘要改編為50~100字X貼文，語氣中立，簡潔吸睛，包含2~3個標籤（如#科技）：\n{state.summary}"
    state.post = llm_call(prompt, max_tokens=100)
    return {"post": state.post}

# 構建工作流程
workflow = StateGraph(SummaryState)
workflow.add_node("summary_node", summary_node)
workflow.add_node("post_node", post_node)
workflow.set_entry_point("summary_node")
workflow.add_edge("summary_node", "post_node")
workflow.add_edge("post_node", END)
graph = workflow.compile()

# 範例新聞（1000字，實際使用可替換）
news = """
人工智慧新突破：醫療影像診斷提升90%準確率
人工智慧（AI）在醫療領域再創里程碑！一項由國際研究團隊開發的新技術，將醫療影像診斷的準確率提升至90%，遠超傳統方法的70%。這項技術利用深度學習模型，結合大數據分析，能自動識別醫療影像中的異常區域，並生成診斷報告。研究顯示，這不僅大幅提升診斷效率，還能將癌症篩查的誤診率降低30%，為患者提供更早的治療機會。團隊表示，這項技術可減輕醫生負擔，讓他們專注於複雜病例的處理。專家認為，AI在醫療影像診斷的應用前景廣闊，未來有望整合至醫院系統，推動智慧醫療發展。然而，技術普及仍需解決資料隱私與模型透明度等挑戰。這項突破已在多個醫學會議上獲得關注，預計將影響全球醫療產業。
[假設此處還有約800字的擴充內容，描述技術細節、案例與未來展望等]
"""

# 執行貼文生成
result = graph.invoke(SummaryState(input=news))
print(f"生成的100字摘要：\n{result['summary']}")
print(f"生成的X貼文：\n{result['post']}")